## Compute Table

In [35]:
import os
from IPython.display import HTML, display

BASELINE = 'pgo-full'
REG_PROFILE = ['dynamic push', 'dynamic pop', 'dynamic spill', 'dynamic reload']
PERF_PROFILE = ['instructions', 'cycles', 'L1-icache-miss', 'L1-dcache-loads', 'L1-dcache-stores', 'time elapsed']
VARANT = ['none'] + ["{}-{}".format(i, j) for i in [1,3,5,10] for j in [10, 20]]
FDOIPRA = [b + 'fdoipra' + (str(i) if i != 1 else "") for i in range(1, 7) for b in ["", "b"]] 
METHODS = [BASELINE] + ['pgo-full-'+v for v in ['ipra']+FDOIPRA]

def load_regprof3(content):
    assert(len(content) == 4)
    result = {}
    for line in content:
        if line.startswith('dynamic push: '):
            result['dynamic push'] = int(line[14:])
        if line.startswith('dynamic pop: '):
            result['dynamic pop'] = int(line[13:])
        if line.startswith('dynamic spill: '):
            result['dynamic spill'] = int(line[15:])
        if line.startswith('dynamic reload: '):
            result['dynamic reload'] = int(line[16:])
    return result

def load_bench(content):
    def get(line, key):
        p = line.find(key)
        if p != -1:
            q = line.find('( +-')
            qe = line[q:].find(')')
            return True, line[:p].strip(), line[q+5:qe-1]
        return False, None, None
    result = {}
    for line in content:
        for c in PERF_PROFILE:
            found, data, error = get(line, c)
            if found:
                if c == 'time elapsed':
                    data = data.split()[0]
                result[c] = (float(data), error)
    return result

class Benchmark:
    def __init__(self, name):
        self.name = name
        self.results = {}
        self.best_reg = {}
        self.best_bench = {}
    
    def load(self, path, cases=['bench', 'regprof3']):
        self.field = cases
        filter = {
            'bench': load_bench,
            'regprof3': load_regprof3
        }
        for case in cases:
            result = {}
            files = [path+'/'+f for f in os.listdir(path) 
                        if os.path.isfile(path+'/'+f)]
            for file in files:
                if file.endswith('.'+case):
                    with open(file, 'r') as f:
                        _ , name = os.path.split(file)
                        result[name[:-len(case)-1]] = filter[case](f.readlines())
            if len(result) > 0:
                self.results[case] = result
        return len(self.results) != 0
    
    def get(self, key, method, var):
        def get_value(value):
            data, error = None, None
            if type(value) is tuple:
                data, error = value
            else:
                data = value
            return data, error
        try:
            index = ''
            if key in REG_PROFILE:
                index = 'regprof3'
            if key in PERF_PROFILE:
                index = 'bench'
            
            if var == 'none':
                return get_value(self.results[index][method][key])
            return get_value(self.results[index][method+'.'+var][key])
        except:
            return None, None

    def better(self, key, method, var):
        base, _ = self.get(key, BASELINE, 'none')
        impl, _ = self.get(key, method, var)
        if base != None and impl != None:
            return (impl - base) * 100 / base
        else:
            return None

    def best(self, key, method):
        if key in REG_PROFILE:
            if method in self.best_reg:
                return self.best_reg[method][key]
            data = []
            for var in VARANT:
                improve = 0
                detail = {}
                for k in REG_PROFILE:
                    if self.better(k, method, var) != None:
                        improve += self.better(k, method, var)
                        detail[k], _ = self.get(k, method, var)
                if len(detail) > 0:
                    data.append((improve, detail))
            if len(data) == 0:
                return None
            _, result = min(data)
            self.best_reg[method] = result
            return self.best_reg[method][key]

    def __str__(self) -> str:
        s = self.name + ': \n'
        for case, data in self.results.items():
            s += case
            for j in data.keys():
                s += ("    " + j)
            s += '\n'
        return s

def load_build_folder(build_folder = '../build/benchmarks/'):
    """Load a build folder and return a list of results"""
    benchmarks = []
    build_folder = os.path.abspath(build_folder)
    print(build_folder)
    folders = [f for f in os.listdir(build_folder) 
                        if os.path.isdir(f)]    
    for folder in folders:
        B = Benchmark(folder)
        if B.load(build_folder+'/'+folder):
            benchmarks.append(B)
    return benchmarks

def printTable(data, header=None, title=None, rowspan=0):
    def gen_row(row, el='td'):
        gen_row.counter += 1
        if gen_row.counter == rowspan:
            gen_row.counter = 0
        tmp = '<'+el+'{}>{}</'+el+'>' 
        tmp2 = '</'+el+'><'+el+'>' 
        return tmp.format(
            ' rowspan="'+str(rowspan)+'"' if gen_row.counter==1 else "", 
            tmp2.join(str(_) for _ in row))
    gen_row.counter = 0 if header == None else -1
    display(HTML('<table><h2>{}</h2><tr>{}</tr></table>'.format(title,
        (gen_row(header, 'th') + '</tr><tr>' if header else '') + 
            '</tr><tr>'.join(gen_row(row) for row in data))
    ))

Benchmarks = load_build_folder()

# for bench in Benchmarks:
#     print(bench)



/home/riple/IPRA-exp/build/benchmarks


In [36]:

def show_regprof(Bench: Benchmark, actions):
    data = []
    header = VARANT 
    for case in FDOIPRA:
        for action in actions:
            row = []
            if action == actions[0]:
                row.append(case)
            row.append(action)
            for var in header:
                better = Bench.better(action, 'pgo-full-'+case, var)
                if better == None:
                    row.append('')
                    continue
                cell, _ = Bench.get(action, 'pgo-full-'+case, var)
                row.append(
                    "{} (<span style='color: {};'>{:+.4}%</span>)".format(
                        cell,
                        'red' if better >= 0 else 'green',
                        better))
            cell = Bench.best(action, 'pgo-full-'+case)
            base, _ = Bench.get(action, BASELINE, 'none')
            if cell==None or base == None:
                row.append('')
            else:
                better = (cell - base) * 100 / base
                row.append(
                        "{} (<span style='color: {};'>{:+.4}%</span>)".format(
                            cell, 
                            'red' if better >= 0 else 'green',
                            better))
            data.append(row)
    printTable(data, ['', ''] + header + ['best'], Bench.name, len(actions))

def show_regprof_summary(title, actions):
    data = []
    header = ['',''] + [Bench.name for Bench in Benchmarks]
    for case in METHODS:
        for action in actions:
            row = []
            if action == actions[0]:
                row.append(case)
            row.append(action)
            for Bench in Benchmarks:
                cell = Bench.best(action, case)
                base, _ = Bench.get(action, BASELINE, 'none')
                if cell==None or base == None:
                    row.append('')
                    continue
                better = (cell - base) * 100 / base
                row.append(
                    "{} (<span style='color: {};'>{:+.4}%</span>)".format(
                        cell, 
                        'red' if better >= 0 else 'green',
                        better))
            data.append(row)
    printTable(data, header, title, len(actions))

## Push/Pop and Spill/Reload

In [37]:
show_regprof_summary("Push and Push Counting", ['dynamic push', 'dynamic pop'])

for bench in Benchmarks:
    show_regprof(bench, ['dynamic push', 'dynamic pop'])

In [38]:
show_regprof_summary("Spill and Reload Bytes", ['dynamic spill', 'dynamic reload'])

for bench in Benchmarks:
    show_regprof(bench, ['dynamic spill', 'dynamic reload'])

## Wall Time and Cache Loads/Stores



In [39]:
show_regprof_summary("Wall Time", ['time elapsed', 'instructions', 'cycles'])

for bench in Benchmarks:
    show_regprof(bench, ['time elapsed', 'instructions', 'cycles'])

In [40]:
show_regprof_summary("Cache Loads/Stores", ['L1-dcache-loads', 'L1-dcache-stores'])

for bench in Benchmarks:
    show_regprof(bench, ['L1-dcache-loads', 'L1-dcache-stores'])

In [41]:
show_regprof_summary("ICache Miss Rate", ['L1-icache-miss'])

for bench in Benchmarks:
    show_regprof(bench, ['L1-icache-miss'])

,,leveldb
pgo-full,L1-icache-miss,
pgo-full-ipra,L1-icache-miss,
pgo-full-fdoipra,L1-icache-miss,
pgo-full-bfdoipra,L1-icache-miss,
pgo-full-fdoipra2,L1-icache-miss,
pgo-full-bfdoipra2,L1-icache-miss,
pgo-full-fdoipra3,L1-icache-miss,
pgo-full-bfdoipra3,L1-icache-miss,
pgo-full-fdoipra4,L1-icache-miss,
pgo-full-bfdoipra4,L1-icache-miss,


,,none,1-10,1-20,3-10,3-20,5-10,5-20,10-10,10-20,best
fdoipra,L1-icache-miss,1766739331.0 (+0.8183%),1785864572.0 (+1.91%),1757889074.0 (+0.3133%),1752293790.0 (-0.006029%),1761246950.0 (+0.5049%),1764555109.0 (+0.6937%),1742059461.0 (-0.59%),1761991811.0 (+0.5474%),1845407516.0 (+5.307%),
bfdoipra,L1-icache-miss,1757593023.0 (+0.2964%),1757309702.0 (+0.2802%),1738301959.0 (-0.8045%),1703112426.0 (-2.813%),1733847152.0 (-1.059%),1731589365.0 (-1.188%),1757218602.0 (+0.275%),1752563643.0 (+0.00937%),1761797236.0 (+0.5363%),
fdoipra2,L1-icache-miss,1749360723.0 (-0.1734%),1752298108.0 (-0.005783%),1794713757.0 (+2.415%),1744171520.0 (-0.4695%),1779691751.0 (+1.557%),1752137429.0 (-0.01495%),1918148881.0 (+9.458%),1750172779.0 (-0.1271%),1776016687.0 (+1.348%),
bfdoipra2,L1-icache-miss,1761085777.0 (+0.4957%),1752421982.0 (+0.001286%),1741120231.0 (-0.6436%),1907425985.0 (+8.847%),1751369455.0 (-0.05878%),1743311708.0 (-0.5186%),2015582669.0 (+15.02%),1738527516.0 (-0.7916%),1739288468.0 (-0.7482%),
fdoipra3,L1-icache-miss,1797040207.0 (+2.547%),1846966889.0 (+5.396%),1806984004.0 (+3.115%),1800491219.0 (+2.744%),1810875648.0 (+3.337%),1891193745.0 (+7.92%),1816824748.0 (+3.676%),1881151161.0 (+7.347%),1846361093.0 (+5.362%),
bfdoipra3,L1-icache-miss,1829399197.0 (+4.394%),1799366768.0 (+2.68%),1838030753.0 (+4.887%),1841685541.0 (+5.095%),1786168569.0 (+1.927%),1810726241.0 (+3.328%),1776827531.0 (+1.394%),1792680082.0 (+2.299%),1874503201.0 (+6.968%),
fdoipra4,L1-icache-miss,1777521538.0 (+1.434%),1758704835.0 (+0.3598%),1785221800.0 (+1.873%),1739769072.0 (-0.7207%),1744805786.0 (-0.4333%),1735028569.0 (-0.9913%),1772681594.0 (+1.157%),1784271035.0 (+1.819%),1915545091.0 (+9.31%),
bfdoipra4,L1-icache-miss,1766366464.0 (+0.797%),1737331426.0 (-0.8599%),1740829019.0 (-0.6603%),1852995549.0 (+5.74%),1790143988.0 (+2.154%),1760968537.0 (+0.489%),1759919537.0 (+0.4291%),1735945327.0 (-0.9389%),1782320512.0 (+1.707%),
fdoipra5,L1-icache-miss,1865498877.0 (+6.454%),1866900294.0 (+6.534%),1876565171.0 (+7.085%),1858765947.0 (+6.07%),1865089341.0 (+6.431%),1962338849.0 (+11.98%),1870914764.0 (+6.763%),1894752967.0 (+8.123%),1884803264.0 (+7.556%),
bfdoipra5,L1-icache-miss,1847593709.0 (+5.432%),1865901777.0 (+6.477%),1861117699.0 (+6.204%),1859533213.0 (+6.114%),1879470980.0 (+7.251%),1851589006.0 (+5.66%),1871908900.0 (+6.82%),1854887585.0 (+5.848%),1872354206.0 (+6.845%),
